# ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎  ‎ ‎ ‎ ‎ Scrapping Data from Fbref

referred : https://smehta.medium.com/scrape-and-create-your-own-beautiful-dataset-from-sports-reference-com-using-beautifulsoup-python-c26d6920684e
   
   
   ‎ ‎‎‎‎  ‎ ‎‎ ‎  ‎ ‎ ‎ ‎ ‎ ‎ ‎ Data- Statbomb via [Fbref](https://fbref.com/en/)


In [1]:
#importing required packages

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv

In [2]:
#getting the required parameters

#standard stat

stats = ["player","nationality","position","team","age","birth_year","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xg_assist_per90","xg_xg_assist_per90","npxg_per90","npxg_xa_per90"]

stats3 = ["players_used","possession","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xg_assist","xg_per90","xg_assist_per90","xg_xg_assist_per90","npxg_per90","npxg_xg_assist_per90"]

#keeping

keepers = ["player","nationality","position","team","age","birth_year","gk_games","gk_games_starts","gk_minutes","gk_goals_against","gk_goals_against_per90","gk_shots_on_target_against","gk_saves","gk_save_pct","gk_wins","gk_ties","gk_losses","gk_clean_sheets","gk_clean_sheets_pct","gk_pens_att","gk_pens_allowed","gk_pens_saved","gk_pens_missed"]

keepers3 = ["players_used","gk_games","gk_games_starts","gk_minutes","gk_goals_against","gk_goals_against_per90","shots_on_target_against","saves","save_pct","gk_wins","gk_ties","gk_losses","gk_clean_sheets","clean_sheets_pct","gk_pens_att","gk_pens_allowed","gk_pens_saved","gk_pens_missed"]

#advance keeping

keepersadv = ["player","nationality","position","team","age","birth_year","minutes_90s","gk_goals_against","pens_allowed","gk_free_kick_goals_against","gk_corner_kick_goals_against","gk_own_goals_against","gk_psxg","psnpxg_per_shot_on_target_against","gk_psxg_net","gk_psxg_net_per90","gk_passes_completed_launched","gk_passes_launched","gk_passes_pct_launched","gk_passes","gk_passes_throws","gk_pct_passes_launched","gk_passes_length_avg","gk_goal_kicks","gk_pct_goal_kicks_launched","gk_goal_kick_length_avg","gk_crosses","gk_crosses_stopped","gk_crosses_stopped_pct","gk_def_actions_outside_pen_area","gk_def_actions_outside_pen_area_per90","gk_avg_distance_def_actions"]

keepersadv2 = ["minutes_90s","gk_goals_against","gk_pens_allowed","gk_free_kick_goals_against","gk_corner_kick_goals_against","gk_own_goals_against","gk_psxg","gk_psnpxg_per_shot_on_target_against","gk_psxg_net","gk_psxg_net_per90","gk_passes_completed_launched","gk_passes_launched","gk_passes_pct_launched","gk_passes","gk_passes_throws","gk_pct_passes_launched","gk_passes_length_avg","gk_goal_kicks","gk_pct_goal_kicks_launched","gk_goal_kick_length_avg","gk_crosses","gk_crosses_stopped","gk_crosses_stopped_pct","gk_def_actions_outside_pen_area","gk_def_actions_outside_pen_area_per90","gk_avg_distance_def_actions"]

#shooting

shooting = ["player","nationality","position","team","age","birth_year","minutes_90s","goals","pens_made","pens_att","shots","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]

shooting2 = ["minutes_90s","goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]

shooting3 = ["goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]

#passing

passing = ["player","nationality","position","team","age","birth_year","minutes_90s","passes_completed","passes","passes_pct","passes_total_distance","passes_progressive_distance","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","passes_pct_long","assists","xa","xa_net","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"]

passing2 = ["passes_completed","passes","passes_pct","passes_total_distance","passes_progressive_distance","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","passes_pct_long","assists","xa","xa_net","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"]

#passes types

passing_types = ["player","nationality","position","team","age","birth_year","minutes_90s","passes","passes_live","passes_dead","passes_free_kicks","through_balls","crosses","corner_kicks","corner_kicks_in","corner_kicks_out","corner_kicks_straight","throw_ins","passes_other_body","passes_completed","passes_offsides","passes_blocked"]

passing_types2 = ["passes","passes_live","passes_dead","passes_free_kicks","through_balls","crosses","corner_kicks","corner_kicks_in","corner_kicks_out","corner_kicks_straight","throw_ins","passes_other_body","passes_completed","passes_offsides","passes_blocked"]

#goal and shot creation

gca = ["player","nationality","position","team","age","birth_year","minutes_90s","sca","sca_per90","sca_passes_live","sca_passes_dead","sca_take_ons","sca_shots","sca_fouled","sca_defense","gca","gca_per90","gca_passes_live","gca_passes_dead","gca_take_ons","gca_shots","gca_fouled","gca_defense"]

gca2 = ["sca","sca_per90","sca_passes_live","sca_passes_dead","sca_dribbles","sca_shots","sca_fouled","gca","gca_per90","gca_passes_live","gca_passes_dead","gca_dribbles","gca_shots","gca_fouled","gca_defense"]

#defensive actions

defense = ["player","nationality","position","team","age","birth_year","minutes_90s","tackles","tackles_won","tackles_def_3rd","tackles_mid_3rd","tackles_att_3rd","challenge_tackles","challenges","challenge_tackles_pct","challenges_lost","blocks","blocked_shots","blocked_shots_saves","blocked_passes","interceptions","tackles_interceptions","clearances","errors"]

defense2 = ["tackles","tackles_won","tackles_def_3rd","tackles_mid_3rd","tackles_att_3rd","dribble_tackles","dribbles_vs","dribble_tackles_pct","dribbled_past","blocks","blocked_shots","blocked_shots_saves","blocked_passes","interceptions","tackles_interceptions","clearances","errors"]

#possession

possession = ["player","nationality","position","team","age","birth_year","minutes_90s","touches","touches_def_pen_area","touches_def_3rd","touches_mid_3rd","touches_att_3rd","touches_att_pen_area","touches_live_ball","take_ons","take_ons_won","take_ons_won_pct","take_ons_tackled","take_ons_tackled_pct","carries","carries_distance","carries_progressive_distance","progressive_carries","carries_into_final_third","carries_into_penalty_area","pass_targets","miscontrols","dispossessed","passes_received","progressive_passes_received"]

possession2 = ["touches","touches_def_pen_area","touches_def_3rd","touches_mid_3rd","touches_att_3rd","touches_att_pen_area","touches_live_ball","dribbles_completed","dribbles","dribbles_completed_pct","players_dribbled_past","nutmegs","carries","carry_distance","carry_progressive_distance","progressive_carries","carries_into_final_third","carries_into_penalty_area","pass_targets","passes_received","progressive_passes_received","miscontrols","dispossessed"]

#minutes played

playingtime = ["player","nationality","position","team","age","birth_year","minutes_90s","games","minutes","minutes_per_game","minutes_pct","games_starts","minutes_per_start","games_subs","minutes_per_sub","unused_subs","points_per_match","on_goals_for","on_goals_against","plus_minus","plus_minus_per90","plus_minus_wowy","on_xg_for","on_xg_against","xg_plus_minus","xg_plus_minus_per90","xg_plus_minus_wowy"]

playingtime2 = ["games","minutes","minutes_per_game","minutes_pct","games_starts","minutes_per_start","games_subs","minutes_per_sub","unused_subs","points_per_match","on_goals_for","on_goals_against","plus_minus","plus_minus_per90","plus_minus_wowy","on_xg_for","on_xg_against","xg_plus_minus","xg_plus_minus_per90","xg_plus_minus_wowy"]

#miscallaneous

misc = ["player","nationality","position","team","age","birth_year","minutes_90s","cards_yellow","cards_red","cards_yellow_red","fouls","fouled","offsides","crosses","interceptions","tackles_won","pens_won","pens_conceded","own_goals","ball_recoveries","aerials_won","aerials_lost","aerials_won_pct"]

misc2 = ["cards_yellow","cards_red","cards_yellow_red","fouls","fouled","offsides","crosses","interceptions","tackles_won","pens_won","pens_conceded","own_goals","ball_recoveries","aerials_won","aerials_lost","aerials_won_pct"]


###    Functions created to scrape required data using beautifulsoup

In [3]:
#Functions to get the data in a dataframe using BeautifulSoup


def get_tables(url, text):
    try:
        res = requests.get(url)
        if res.status_code == 200:
            comm = re.compile("<!--|-->")
            soup = BeautifulSoup(comm.sub("", res.text), 'html.parser')
            all_tables = soup.findAll("tbody")
            team_table = all_tables[0]
            team_vs_table = all_tables[1]
            player_table = all_tables[2]

            if text == 'for':
                return player_table, team_table
            if text == 'vs':
                return player_table, team_vs_table
        else:
            print(f"Failed to fetch data. Status code: {res.status_code}")
            return None, None  # Return None to indicate failure
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None  # Return None to indicate failure


def get_frame(features, player_table):
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all('tr')
    for row in rows_player:
        if(row.find('th',{"scope":"row"}) != None):

            for f in features_wanted_player:
                cell = row.find("td",{"data-stat": f})
                if cell is not None:
                    a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='team')&(f!='age')&(f!='birth_year')):
                    text = float(text.replace(',',''))
                if f in pre_df_player:
                    pre_df_player[f].append(text)
                else:
                    pre_df_player[f] = [text]
    df_player = pd.DataFrame.from_dict(pre_df_player)
    return df_player

def get_frame_team(features, team_table):
    pre_df_team = dict()
    #Note: features does not contain team name, it requires special treatment
    features_wanted_team = features
    rows_team = team_table.find_all('tr')
    for row in rows_team:
        if(row.find('th',{"scope":"row"}) != None):
            name = row.find('th',{"data-stat":"team"}).text.strip().encode().decode("utf-8")
            if 'team' in pre_df_team:
                pre_df_team['team'].append(name)
            else:
                pre_df_team['team'] = [name]
            for f in features_wanted_team:
                cell = row.find("td",{"data-stat": f})
                if cell is not None:
                    a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='team')&(f!='age')&(f!='birth_year')):
                    text = float(text.replace(',',''))
                if f in pre_df_team:
                    pre_df_team[f].append(text)
                else:
                    pre_df_team[f] = [text]
    df_team = pd.DataFrame.from_dict(pre_df_team)
    return df_team

def frame_for_category(category,top,end,features):
    url = (top + category + end)
    player_table, team_table = get_tables(url,'for')
    df_player = get_frame(features, player_table)
    return df_player






#Function to get the player data for outfield player, includes all categories - standard stats, shooting
#passing, passing types, goal and shot creation, defensive actions, possession, and miscallaneous
def get_outfield_data(top, end):
    df1 = frame_for_category('stats',top,end,stats)
    df2 = frame_for_category('shooting',top,end,shooting2)
    df3 = frame_for_category('passing',top,end,passing2)
    df4 = frame_for_category('passing_types',top,end,passing_types2)
    df5 = frame_for_category('gca',top,end,gca2)
    df6 = frame_for_category('defense',top,end,defense2)
    df7 = frame_for_category('possession',top,end,possession2)
    df8 = frame_for_category('misc',top,end,misc2)
    df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df


#Function to get keeping and advance goalkeeping data
def get_keeper_data(top,end):
    df1 = frame_for_category('keepers',top,end,keepers)
    df2 = frame_for_category('keepersadv',top,end,keepersadv2)
    df = pd.concat([df1, df2], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df






In [4]:
# to scrape data of premier league outfield players

#2022-2023
df_ploutfield2022 = get_outfield_data('https://fbref.com/en/comps/9/2022-2023/','/2022-2023-Premier-League-Stats')


df_ploutfield2022

,player,nationality,position,team,age,birth_year,games,games_starts,minutes,goals,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Brenden Aaronson,us USA,"MF,FW",Leeds United,21,2000,36.0,28.0,2372.0,1.0,...,16.0,55.0,6.0,0.0,0.0,0.0,128.0,8.0,39.0,17.0
1,George Abbott,eng ENG,MF,Tottenham,16,2005,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Che Adams,sct SCO,FW,Southampton,26,1996,28.0,23.0,1992.0,5.0,...,38.0,27.0,14.0,0.0,0.0,0.0,53.0,61.0,103.0,37.2
3,Tyler Adams,us USA,MF,Leeds United,23,1999,24.0,24.0,2156.0,0.0,...,37.0,21.0,1.0,0.0,0.0,0.0,199.0,25.0,15.0,62.5
4,Tosin Adarabioyo,eng ENG,DF,Fulham,24,1997,25.0,23.0,2085.0,1.0,...,17.0,5.0,3.0,0.0,0.0,0.0,110.0,59.0,26.0,69.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,Jordan Zemura,zw ZIM,DF,Bournemouth,22,1999,19.0,17.0,1454.0,0.0,...,17.0,14.0,0.0,0.0,1.0,0.0,78.0,7.0,11.0,38.9
565,Oleksandr Zinchenko,ua UKR,DF,Arsenal,25,1996,27.0,26.0,2118.0,1.0,...,6.0,11.0,1.0,0.0,0.0,0.0,163.0,51.0,20.0,71.8
566,Hakim Ziyech,ma MAR,"FW,MF",Chelsea,29,1993,18.0,6.0,661.0,0.0,...,12.0,14.0,3.0,0.0,0.0,0.0,46.0,0.0,5.0,0.0
567,Kurt Zouma,fr FRA,DF,West Ham,27,1994,25.0,24.0,1989.0,2.0,...,14.0,7.0,0.0,0.0,0.0,0.0,84.0,55.0,13.0,80.9


In [5]:
# to scrape data of la liga outfield players


#2022-2023
df_LLoutfield2022 = get_outfield_data('https://fbref.com/en/comps/12/2022-2023/','/2022-2023-La-Liga-Stats')

df_LLoutfield2022

,player,nationality,position,team,age,birth_year,games,games_starts,minutes,goals,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Abner,br BRA,DF,Betis,22,2000,13.0,7.0,607.0,0.0,...,8.0,2.0,0.0,0.0,1.0,0.0,43.0,9.0,16.0,36.0
1,Álvaro Aceves,es ESP,GK,Valladolid,19,2003,1.0,0.0,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,50.0
2,Marcos Acuña,ar ARG,DF,Sevilla,30,1991,30.0,21.0,1912.0,3.0,...,32.0,34.0,1.0,0.0,1.0,0.0,141.0,30.0,18.0,62.5
3,Julen Agirrezabala,es ESP,GK,Athletic Club,21,2000,8.0,7.0,675.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.0,1.0,0.0,100.0
4,Álvaro Aguado,es ESP,MF,Valladolid,26,1996,34.0,18.0,1888.0,1.0,...,13.0,33.0,0.0,0.0,1.0,0.0,130.0,15.0,17.0,46.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,Oier Zarraga,es ESP,MF,Athletic Club,23,1999,26.0,6.0,852.0,0.0,...,17.0,12.0,0.0,0.0,0.0,0.0,56.0,9.0,12.0,42.9
592,Igor Zubeldia,es ESP,DF,Real Sociedad,25,1997,31.0,29.0,2542.0,1.0,...,44.0,10.0,1.0,0.0,0.0,1.0,165.0,60.0,53.0,53.1
593,Martín Zubimendi,es ESP,MF,Real Sociedad,23,1999,36.0,35.0,3101.0,1.0,...,50.0,31.0,1.0,0.0,1.0,0.0,218.0,58.0,34.0,63.0
594,Blanco,0,MF,Cádiz,22,2000,3.0,2.0,158.0,0.0,...,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# to scrape data of Serie A outfield players

#2022-2023
df_SAoutfield2022 = get_outfield_data('https://fbref.com/en/comps/11/2022-2023/','/2022-2023-Serie-A-Stats')

df_SAoutfield2022

,player,nationality,position,team,age,birth_year,games,games_starts,minutes,goals,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,James Abankwah,ie IRL,DF,Udinese,18,2004,2.0,1.0,63.0,0.0,...,2.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,100.0
1,Oliver Abildgaard,dk DEN,MF,Hellas Verona,26,1996,13.0,6.0,592.0,0.0,...,13.0,7.0,0.0,0.0,0.0,0.0,37.0,50.0,21.0,70.4
2,Tammy Abraham,eng ENG,FW,Roma,24,1997,38.0,24.0,2189.0,8.0,...,33.0,48.0,11.0,1.0,0.0,0.0,46.0,80.0,74.0,51.9
3,Christian Acella,it ITA,MF,Cremonese,20,2002,1.0,0.0,15.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,Francesco Acerbi,it ITA,DF,Inter,34,1988,31.0,25.0,2425.0,0.0,...,17.0,13.0,2.0,0.0,0.0,0.0,151.0,78.0,42.0,65.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,Petar Zovko,ba BIH,GK,Spezia,20,2002,1.0,0.0,74.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
599,Szymon Żurkowski,pl POL,MF,Fiorentina,24,1997,2.0,0.0,32.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,50.0
600,Szymon Żurkowski,pl POL,MF,Spezia,24,1997,10.0,3.0,427.0,0.0,...,15.0,11.0,0.0,0.0,0.0,0.0,26.0,6.0,10.0,37.5
601,Milan Đurić,ba BIH,FW,Hellas Verona,32,1990,28.0,14.0,1366.0,1.0,...,24.0,29.0,5.0,0.0,0.0,0.0,30.0,197.0,57.0,77.6


In [7]:
# to scrape data of Bundesliga outfield players
#2022-2023
df_BDoutfield2022 = get_outfield_data('https://fbref.com/en/comps/20/2022-2023/','/2022-2023-Bundesliga-Stats')

df_BDoutfield2022

,player,nationality,position,team,age,birth_year,games,games_starts,minutes,goals,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Paxten Aaronson,us USA,"MF,DF",Eint Frankfurt,18,2003,7.0,0.0,173.0,0.0,...,5.0,3.0,1.0,0.0,0.0,0.0,8.0,1.0,1.0,50.0
1,Sargis Adamyan,am ARM,"FW,MF",Köln,29,1993,24.0,4.0,644.0,1.0,...,17.0,6.0,4.0,0.0,0.0,0.0,40.0,12.0,25.0,32.4
2,Karim Adeyemi,de GER,"FW,MF",Dortmund,20,2002,24.0,20.0,1386.0,6.0,...,21.0,30.0,1.0,0.0,0.0,0.0,66.0,10.0,4.0,71.4
3,Amine Adli,fr FRA,"FW,MF",Leverkusen,22,2000,26.0,16.0,1435.0,5.0,...,39.0,47.0,4.0,2.0,1.0,0.0,61.0,29.0,41.0,41.4
4,Felix Agu,de GER,"DF,FW",Werder Bremen,22,1999,3.0,0.0,49.0,0.0,...,3.0,3.0,0.0,0.0,1.0,0.0,3.0,1.0,1.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,Rodrigo Zalazar,uy URU,"MF,FW",Schalke 04,22,1999,22.0,15.0,1321.0,1.0,...,22.0,36.0,4.0,0.0,0.0,0.0,97.0,24.0,39.0,38.1
511,Aaron Zehnter,de GER,DF,Augsburg,17,2004,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512,Robin Zentner,de GER,GK,Mainz 05,27,1994,26.0,26.0,2340.0,0.0,...,0.0,3.0,0.0,0.0,0.0,0.0,43.0,16.0,0.0,100.0
513,Michael Zetterer,de GER,GK,Werder Bremen,27,1995,2.0,1.0,152.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,100.0


In [8]:
# to scrape data of Ligue 1 outfield players

#2022-2023
df_L1outfield2022 = get_outfield_data('https://fbref.com/en/comps/13/2022-2023/','/2022-2023-Ligue-1-Stats')

df_L1outfield2022



,player,nationality,position,team,age,birth_year,games,games_starts,minutes,goals,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Yunis Abdelhamid,ma MAR,DF,Reims,34,1987,37.0,37.0,3330.0,1.0,...,49.0,28.0,2.0,0.0,0.0,0.0,263.0,93.0,51.0,64.6
1,Himad Abdelli,dz ALG,"MF,FW",Angers,22,1999,30.0,24.0,2137.0,2.0,...,43.0,31.0,1.0,0.0,0.0,0.0,180.0,18.0,23.0,43.9
2,Salis Abdul Samed,gh GHA,MF,Lens,22,2000,33.0,33.0,2894.0,1.0,...,71.0,38.0,0.0,0.0,0.0,1.0,213.0,14.0,17.0,45.2
3,Laurent Abergel,fr FRA,MF,Lorient,29,1993,29.0,28.0,2342.0,0.0,...,28.0,29.0,0.0,0.0,0.0,0.0,193.0,8.0,15.0,34.8
4,Matthis Abline,fr FRA,"FW,MF",Auxerre,19,2003,19.0,8.0,806.0,2.0,...,14.0,10.0,5.0,1.0,0.0,0.0,13.0,12.0,30.0,28.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,Mehdi Zeffane,dz ALG,"DF,MF",Clermont Foot,30,1992,18.0,7.0,658.0,1.0,...,4.0,3.0,1.0,0.0,0.0,0.0,36.0,3.0,8.0,27.3
602,Arber Zeneli,xk KVX,"FW,MF",Reims,27,1995,23.0,12.0,1000.0,0.0,...,7.0,12.0,3.0,1.0,0.0,0.0,58.0,4.0,8.0,33.3
603,Nathan Zeze,fr FRA,DF,Nantes,17,2005,1.0,1.0,59.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,50.0
604,Edon Zhegrova,xk KVX,"FW,MF",Lille,23,1999,22.0,11.0,1056.0,3.0,...,11.0,15.0,2.0,1.0,0.0,0.0,58.0,0.0,6.0,0.0


In [9]:
#to concatenate all the league outfield players data in Top 5 league

df_alloutfieldplayers2022_2023 = pd.concat([df_ploutfield2022,df_LLoutfield2022,df_SAoutfield2022,df_BDoutfield2022,df_L1outfield2022])

df_alloutfieldplayers2022_2023.to_csv('All_Outfield2022_2023.csv',index=False)
df_alloutfieldplayers2022_2023

,player,nationality,position,team,age,birth_year,games,games_starts,minutes,goals,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Brenden Aaronson,us USA,"MF,FW",Leeds United,21,2000,36.0,28.0,2372.0,1.0,...,16.0,55.0,6.0,0.0,0.0,0.0,128.0,8.0,39.0,17.0
1,George Abbott,eng ENG,MF,Tottenham,16,2005,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Che Adams,sct SCO,FW,Southampton,26,1996,28.0,23.0,1992.0,5.0,...,38.0,27.0,14.0,0.0,0.0,0.0,53.0,61.0,103.0,37.2
3,Tyler Adams,us USA,MF,Leeds United,23,1999,24.0,24.0,2156.0,0.0,...,37.0,21.0,1.0,0.0,0.0,0.0,199.0,25.0,15.0,62.5
4,Tosin Adarabioyo,eng ENG,DF,Fulham,24,1997,25.0,23.0,2085.0,1.0,...,17.0,5.0,3.0,0.0,0.0,0.0,110.0,59.0,26.0,69.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601,Mehdi Zeffane,dz ALG,"DF,MF",Clermont Foot,30,1992,18.0,7.0,658.0,1.0,...,4.0,3.0,1.0,0.0,0.0,0.0,36.0,3.0,8.0,27.3
602,Arber Zeneli,xk KVX,"FW,MF",Reims,27,1995,23.0,12.0,1000.0,0.0,...,7.0,12.0,3.0,1.0,0.0,0.0,58.0,4.0,8.0,33.3
603,Nathan Zeze,fr FRA,DF,Nantes,17,2005,1.0,1.0,59.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,50.0
604,Edon Zhegrova,xk KVX,"FW,MF",Lille,23,1999,22.0,11.0,1056.0,3.0,...,11.0,15.0,2.0,1.0,0.0,0.0,58.0,0.0,6.0,0.0


####  ‎ ‎‎ ‎ ‎ ‎ ‎ ‎  ‎ ‎‎ ‎ ‎ ‎ ‎  ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎  ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎   ‎‎ ‎ ‎ ‎ ‎ ‎  ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎‎ ‎ ‎ ‎ ‎ ‎  ‎ ‎‎ ‎ ‎ ‎ ‎ ‎  ‎  ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ ‎  ‎ ‎‎ ‎ ‎ ‎ ‎ ‎ To scrape data of keepers


In [10]:
# to scrape data of premier league keepers

#2022-2023
df_plkeeper2022 = get_keeper_data('https://fbref.com/en/comps/9/2022-2023/','/2022-2023-Premier-League-Stats')

df_plkeeper2022

,player,nationality,position,team,age,birth_year,gk_games,gk_games_starts,gk_minutes,gk_goals_against,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,Alisson,br BRA,GK,Liverpool,29,1992,37.0,37.0,3330.0,43.0,...,26.9,193.0,26.9,31.3,397.0,23.0,5.8,89.0,2.41,19.8
1,Alphonse Areola,fr FRA,GK,West Ham,29,1993,5.0,2.0,309.0,7.0,...,33.4,30.0,43.3,33.0,47.0,2.0,4.3,1.0,0.29,10.3
2,Kepa Arrizabalaga,es ESP,GK,Chelsea,27,1994,29.0,29.0,2565.0,33.0,...,26.3,182.0,35.7,31.5,332.0,15.0,4.5,52.0,1.82,16.9
3,Gavin Bazunu,ie IRL,GK,Southampton,20,2002,32.0,32.0,2880.0,56.0,...,38.1,237.0,67.5,50.0,456.0,36.0,7.9,48.0,1.50,15.1
4,Asmir Begović,ba BIH,GK,Everton,35,1987,1.0,1.0,90.0,0.0,...,34.9,8.0,62.5,46.8,23.0,1.0,4.3,0.0,0.00,5.0
5,Daniel Bentley,eng ENG,GK,Wolves,29,1993,2.0,2.0,180.0,3.0,...,32.9,23.0,30.4,32.2,37.0,5.0,13.5,1.0,0.50,8.3
6,Martin Dúbravka,sk SVK,GK,Newcastle Utd,33,1989,2.0,1.0,157.0,1.0,...,20.7,15.0,40.0,35.3,30.0,1.0,3.3,0.0,0.00,12.4
7,Ederson,br BRA,GK,Manchester City,28,1993,35.0,35.0,3150.0,32.0,...,28.0,152.0,30.9,32.3,286.0,19.0,6.6,33.0,0.94,15.7
8,Łukasz Fabiański,pl POL,GK,West Ham,37,1985,36.0,36.0,3111.0,48.0,...,39.3,249.0,59.8,44.8,513.0,29.0,5.7,25.0,0.72,13.0
9,Fraser Forster,eng ENG,GK,Tottenham,34,1988,14.0,13.0,1215.0,24.0,...,30.2,93.0,49.5,37.6,193.0,5.0,2.6,16.0,1.19,15.2


In [11]:
# to scrape data of la liga keeper


#2022-2023
df_LLkeeper2022 = get_keeper_data('https://fbref.com/en/comps/12/2022-2023/','/2022-2023-La-Liga-Stats')

df_LLkeeper2022

,player,nationality,position,team,age,birth_year,gk_games,gk_games_starts,gk_minutes,gk_goals_against,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,Álvaro Aceves,es ESP,GK,Valladolid,19,2003,1.0,0.0,13.0,0.0,...,25.7,1.0,0.0,24.0,1.0,0.0,0.0,2.0,13.85,33.0
1,Julen Agirrezabala,es ESP,GK,Athletic Club,21,2000,8.0,7.0,675.0,6.0,...,30.5,62.0,77.4,55.4,96.0,11.0,11.5,10.0,1.33,15.4
2,Sergio Asenjo,es ESP,GK,Valladolid,33,1989,10.0,10.0,900.0,21.0,...,35.8,92.0,66.3,51.2,166.0,5.0,3.0,17.0,1.70,16.1
3,Édgar Badía,es ESP,GK,Elche,30,1992,36.0,36.0,3240.0,63.0,...,34.2,296.0,54.4,41.7,574.0,21.0,3.7,20.0,0.56,12.7
4,Yassine Bounou,ma MAR,GK,Sevilla,31,1991,25.0,25.0,2151.0,41.0,...,30.9,162.0,51.9,42.1,308.0,20.0,6.5,39.0,1.63,17.4
5,Claudio Bravo,cl CHI,GK,Betis,39,1983,12.0,12.0,1080.0,9.0,...,39.6,101.0,75.2,52.5,213.0,4.0,1.9,6.0,0.50,12.1
6,Juan Carlos,es ESP,GK,Girona,34,1988,10.0,10.0,900.0,18.0,...,28.7,62.0,24.2,30.3,120.0,6.0,5.0,21.0,2.10,16.9
7,Thibaut Courtois,be BEL,GK,Real Madrid,30,1992,31.0,31.0,2790.0,29.0,...,27.7,152.0,21.7,29.5,366.0,32.0,8.7,40.0,1.29,15.4
8,Stole Dimitrievski,mk MKD,GK,Rayo Vallecano,28,1993,37.0,37.0,3324.0,49.0,...,38.6,299.0,56.9,46.7,500.0,24.0,4.8,52.0,1.41,16.1
9,Marko Dmitrović,rs SRB,GK,Sevilla,30,1992,15.0,13.0,1269.0,13.0,...,44.0,131.0,84.0,62.4,216.0,12.0,5.6,10.0,0.71,12.2


In [12]:
# to scrape data of Serie A keeper

#2022-2023
df_SAkeeper2022 = get_keeper_data('https://fbref.com/en/comps/11/2022-2023/','/2022-2023-Serie-A-Stats')

df_SAkeeper2022

,player,nationality,position,team,age,birth_year,gk_games,gk_games_starts,gk_minutes,gk_goals_against,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,Emil Audero,it ITA,GK,Sampdoria,25,1997,25.0,25.0,2250.0,39.0,...,39.1,200.0,57.0,44.2,356.0,24.0,6.7,23.0,0.92,13.9
1,Francesco Bardi,it ITA,GK,Bologna,30,1992,1.0,1.0,90.0,0.0,...,35.6,9.0,44.4,33.4,10.0,0.0,0.0,0.0,0.00,0.0
2,Marco Carnesecchi,it ITA,GK,Cremonese,22,2000,27.0,27.0,2430.0,47.0,...,41.4,203.0,66.0,53.1,402.0,29.0,7.2,29.0,1.07,14.1
3,Michele Cerofolini,it ITA,GK,Fiorentina,23,1999,5.0,5.0,450.0,3.0,...,33.3,28.0,42.9,38.1,29.0,1.0,3.4,3.0,0.60,13.8
4,Andrea Consigli,it ITA,GK,Sassuolo,35,1987,35.0,35.0,3150.0,55.0,...,33.3,285.0,31.2,32.5,467.0,28.0,6.0,28.0,0.80,14.2
5,Alex Cordaz,it ITA,GK,Inter,39,1983,1.0,0.0,26.0,0.0,...,33.5,5.0,60.0,52.0,7.0,0.0,0.0,0.0,0.00,0.0
6,Alessio Cragno,it ITA,GK,Monza,28,1994,1.0,1.0,90.0,3.0,...,22.6,6.0,16.7,33.3,7.0,0.0,0.0,2.0,2.00,25.7
7,Michele Di Gregorio,it ITA,GK,Monza,25,1997,37.0,37.0,3330.0,49.0,...,32.2,247.0,37.2,34.9,541.0,14.0,2.6,32.0,0.86,13.4
8,Bartłomiej Drągowski,pl POL,GK,Spezia,24,1997,34.0,34.0,2946.0,54.0,...,34.6,196.0,58.7,46.5,472.0,13.0,2.8,32.0,0.98,14.1
9,Wladimiro Falcone,it ITA,GK,Lecce,27,1995,38.0,38.0,3420.0,46.0,...,42.1,282.0,79.1,53.5,534.0,23.0,4.3,42.0,1.11,14.1


In [13]:
# to scrape data of Bundesliga keeper
#2022-2023
df_BDkeeper2022 = get_keeper_data('https://fbref.com/en/comps/20/2022-2023/','/2022-2023-Bundesliga-Stats')

df_BDkeeper2022

,player,nationality,position,team,age,birth_year,gk_games,gk_games_starts,gk_minutes,gk_goals_against,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,Oliver Baumann,de GER,GK,Hoffenheim,32,1990,34.0,34.0,3060.0,57.0,...,33.6,249.0,45.8,39.3,529.0,43.0,8.1,43.0,1.26,13.4
1,Janis Blaswich,de GER,GK,RB Leipzig,31,1991,26.0,26.0,2340.0,29.0,...,25.3,156.0,26.9,30.0,288.0,20.0,6.9,39.0,1.50,17.0
2,Fabian Bredlow,de GER,GK,Stuttgart,27,1995,15.0,15.0,1350.0,23.0,...,27.6,91.0,29.7,33.5,203.0,12.0,5.9,15.0,1.00,15.2
3,Koen Casteels,be BEL,GK,Wolfsburg,30,1992,34.0,34.0,3060.0,48.0,...,33.7,258.0,57.0,47.5,489.0,29.0,5.9,51.0,1.50,16.5
4,Oliver Christensen,dk DEN,GK,Hertha BSC,23,1999,33.0,33.0,2970.0,68.0,...,35.9,235.0,72.8,54.3,486.0,33.0,6.8,70.0,2.12,18.8
5,Finn Dahmen,de GER,GK,Mainz 05,24,1998,8.0,8.0,720.0,16.0,...,36.6,54.0,77.8,54.5,158.0,17.0,10.8,8.0,1.00,13.8
6,Tjark Ernst,de GER,GK,Hertha BSC,19,2003,1.0,1.0,90.0,1.0,...,31.2,13.0,53.8,43.1,18.0,0.0,0.0,0.0,0.00,3.5
7,Ralf Fährmann,de GER,GK,Schalke 04,33,1988,12.0,12.0,1025.0,13.0,...,36.8,83.0,67.5,48.9,162.0,11.0,6.8,9.0,0.79,11.6
8,Mark Flekken,nl NED,GK,Freiburg,29,1993,34.0,34.0,3060.0,44.0,...,37.6,231.0,73.6,58.0,434.0,32.0,7.4,43.0,1.26,15.0
9,Rafał Gikiewicz,pl POL,GK,Augsburg,34,1987,23.0,23.0,2070.0,41.0,...,33.0,234.0,62.8,48.9,363.0,21.0,5.8,45.0,1.96,16.7


In [14]:
# to scrape data of Ligue 1 keeper

#2022-2023
df_L1keeper2022 = get_keeper_data('https://fbref.com/en/comps/13/2022-2023/','/2022-2023-Ligue-1-Stats')

df_L1keeper2022


,player,nationality,position,team,age,birth_year,gk_games,gk_games_starts,gk_minutes,gk_goals_against,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,Doğan Alemdar,tr TUR,GK,Rennes,19,2002,6.0,4.0,406.0,8.0,...,33.0,35.0,71.4,51.4,75.0,5.0,6.7,5.0,1.11,14.8
1,Paul Bernardoni,fr FRA,GK,Angers,25,1997,27.0,27.0,2430.0,62.0,...,34.6,185.0,55.1,42.6,357.0,21.0,5.9,18.0,0.67,11.4
2,Marco Bizot,nl NED,GK,Brest,31,1991,37.0,37.0,3330.0,54.0,...,35.5,289.0,58.5,43.9,520.0,30.0,5.8,25.0,0.68,11.6
3,Rubén Blanco,es ESP,GK,Marseille,27,1995,6.0,6.0,495.0,6.0,...,27.4,41.0,31.7,32.1,73.0,3.0,4.1,7.0,1.27,17.9
4,Joaquín Blázquez,ar ARG,GK,Brest,21,2001,1.0,1.0,90.0,0.0,...,45.7,11.0,18.2,26.3,11.0,2.0,18.2,0.0,0.00,3.5
5,Marcin Bułka,pl POL,GK,Nice,22,1999,2.0,2.0,180.0,2.0,...,28.1,18.0,55.6,44.2,26.0,2.0,7.7,3.0,1.50,13.8
6,Matis Carvalho,pt POR,GK,Montpellier,23,1999,2.0,1.0,136.0,3.0,...,49.4,14.0,92.9,61.4,38.0,2.0,5.3,0.0,0.00,8.3
7,Leonardo César Jardim,br BRA,GK,Lille,27,1995,6.0,6.0,540.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
8,Lucas Chevalier,fr FRA,GK,Lille,20,2001,32.0,32.0,2880.0,31.0,...,29.4,105.0,42.9,37.2,317.0,22.0,6.9,25.0,0.78,13.5
9,Benoît Costil,fr FRA,GK,Auxerre,35,1987,19.0,19.0,1710.0,41.0,...,36.8,137.0,74.5,51.7,285.0,3.0,1.1,11.0,0.58,13.1


In [15]:
#to concatenate all the league keeper data in Top 5 league

df_allkeeper2022_2023 = pd.concat([df_plkeeper2022,df_LLkeeper2022,df_SAkeeper2022,df_BDkeeper2022,df_L1keeper2022])

df_allkeeper2022_2023.to_csv('All_keeper.csv',index=False)

df_allkeeper2022_2023

,player,nationality,position,team,age,birth_year,gk_games,gk_games_starts,gk_minutes,gk_goals_against,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,Alisson,br BRA,GK,Liverpool,29,1992,37.0,37.0,3330.0,43.0,...,26.9,193.0,26.9,31.3,397.0,23.0,5.8,89.0,2.41,19.8
1,Alphonse Areola,fr FRA,GK,West Ham,29,1993,5.0,2.0,309.0,7.0,...,33.4,30.0,43.3,33.0,47.0,2.0,4.3,1.0,0.29,10.3
2,Kepa Arrizabalaga,es ESP,GK,Chelsea,27,1994,29.0,29.0,2565.0,33.0,...,26.3,182.0,35.7,31.5,332.0,15.0,4.5,52.0,1.82,16.9
3,Gavin Bazunu,ie IRL,GK,Southampton,20,2002,32.0,32.0,2880.0,56.0,...,38.1,237.0,67.5,50.0,456.0,36.0,7.9,48.0,1.50,15.1
4,Asmir Begović,ba BIH,GK,Everton,35,1987,1.0,1.0,90.0,0.0,...,34.9,8.0,62.5,46.8,23.0,1.0,4.3,0.0,0.00,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,Rémy Riou,fr FRA,GK,Lyon,34,1987,7.0,6.0,601.0,8.0,...,28.7,47.0,19.1,26.0,78.0,6.0,7.7,10.0,1.50,17.2
37,Brice Samba,fr FRA,GK,Lens,28,1994,37.0,37.0,3330.0,28.0,...,33.3,160.0,33.1,32.7,384.0,34.0,8.9,41.0,1.11,15.4
38,Kasper Schmeichel,dk DEN,GK,Nice,35,1986,36.0,36.0,3240.0,35.0,...,32.7,278.0,39.2,35.7,455.0,24.0,5.3,23.0,0.64,12.9
39,Matz Sels,be BEL,GK,Strasbourg,30,1992,38.0,38.0,3420.0,59.0,...,36.9,196.0,55.1,43.9,504.0,31.0,6.2,32.0,0.84,12.8
